In [ ]:
from netsuitesdk import NetSuiteConnection
from zeep.helpers import serialize_object

import os
from dotenv import load_dotenv

import pandas as pd
import json
from datetime import datetime
import time
from http.client import RemoteDisconnected

from transaction_helper import *

In [ ]:
load_dotenv()

In [ ]:
NETSUITE_CONFIG = {
    'account': os.getenv('account'),
    'consumer_key': os.getenv('consumer_key'),
    'consumer_secret': os.getenv('consumer_secret'),
    'token_key': os.getenv('token_key'),
    'token_secret': os.getenv('token_secret'),
    'wsdl_version': os.getenv('wsdl_version')
}

In [ ]:
ns_connection = NetSuiteConnection(
    account=NETSUITE_CONFIG['account'],
    consumer_key=NETSUITE_CONFIG['consumer_key'],
    consumer_secret=NETSUITE_CONFIG['consumer_secret'],
    token_key=NETSUITE_CONFIG['token_key'],
    token_secret=NETSUITE_CONFIG['token_secret']

)

In [ ]:
sdk_entities = [attr for attr in dir(ns_connection) if '__' not in attr] # object instantiated in NetSuiteConnection attributes are covered entities by this sdk 
print(f"Total Entities covered by SDK: {len(sdk_entities)} \n{sdk_entities}")

In [ ]:
ls_counts = []
for entity in sdk_entities:
    dict = {
        'entity': entity,
        'count': None,
        'error': None
    }

    entity_obj = getattr(ns_connection, entity)
    try:
        count = entity_obj.count()
        dict.update({'count': count})
        ls_counts.append(dict)
    except Exception as e:
        dict.update({'error': e})
        ls_counts.append(dict)

df_count = pd.DataFrame(ls_counts)
df_count

In [ ]:
ns_connection.vendor_bills.count()

### Fetching non registers search objects in SEARCH_RECORD_TYPES of SDK

In [ ]:
def stringify(value):
    if isinstance(value, (dict, list)):
        return json.dumps(value, default=str)
    elif isinstance(value, datetime):
        return value.isoformat()
    return value

In [ ]:
# get_invoices(ns_connection), get_credit_memos(ns_connection), get_vendor_credits(ns_connection)
resp_zeep = get_custom_lists(ns_connection)
print(f"No of records fetched: {len(resp_zeep)}")

# Convert zeep objects to dicts
resp_dict = [serialize_object(row) for row in resp_zeep]

processed_data = [{k: stringify(v) for k, v in row.items()} for row in resp_dict]

df_data = pd.DataFrame(processed_data)
display(df_data.head())

entity_name = 'custom_lists' # invoices, credit_memos
df_data.to_csv(f"netsuite_fetched_data/{entity_name}.csv", index=False)

In [ ]:
diagnose_available_types(ns_connection)

In [ ]:
journal_entries = get_journal_entries(ns_connection)
print(len(journal_entries))
print(type(journal_entries))
journal_entries[:1]

In [ ]:
custom_segments = get_custom_segments(ns_connection)
print(len(custom_segments))
print(type(custom_segments))
custom_segments[:1]

In [ ]:
usages = get_usages(ns_connection)
print(len(usages))
print(type(usages))
usages[:1]